# Deploying a Lookup Analytic in the Browser

In this notebook, we will show how to create a `.air` file to perform a simple lookup of analytic results from an analytic that has already been run.  To do this, we will create a simple example which will utilize a regular expression to identify strings of digits which could potentially be phone numbers.  Once those are identified, the phone numbers will be stripped of all punctuation (characters such as parentheses and hyphens) and looked up in a fictional customer database to see if there is any additional information about that customer.

## Dependencies

For this notebook, the following dependencies are required:

- `aisquared`

This package is avilable on on [pypi](https://pypi.org) via `pip`.  The following cell also runs the commands to install this dependency as well as imports them into the notebook environment.

In [2]:
! pip install aisquared

import aisquared
import json

## Analytic Creation

Now that the required packages have been installed and imported, it is time to create the results of the analytic.  These results are going to be recorded in JSON form via a Python dictionary, with the top-level key being the digits of the phone number and the lower-level keys being the associated individual's name and customer ID.

This dictionary will then be saved to a JSON file, and we will then configure the `.air` file configuration around it.

In [3]:
# Configure the example results

results = {
    '1111111111' : {
        'name' : 'John Doe',
        'customerID' : 11111
    },
    '2222222222' : {
        'name' : 'Jane Doe',
        'customerID' : 22222
    },
    '3333333333' : {
        'name' : 'Alice Doe',
        'customerID' : 33333
    },
    '4444444444' : {
        'name' : 'Bob Doe',
        'customerID' : 44444
    },
    '5555555555' : {
        'name' : 'Charlie Doe',
        'customerID' : 55555
    },
    '6666666666' : {
        'name' : 'Denise Doe',
        'customerID' : 66666
    },
    '7777777777' : {
        'name' : 'Evan Doe',
        'customerID' : 77777
    },
    '8888888888' : {
        'name' : 'Frank Doe',
        'customerID' : 88888
    },
    '9999999999' : {
        'name' : 'Gregory Doe',
        'customerID' : 99999
    }
}

# Save the analytic as a JSON file
file_name = 'analytic.json'
with open(file_name, 'w') as f:
    json.dump(results, f)

## Create the ModelConfiguration

In the following cell, we configure the harvesting, preprocessing, analytic, postprocessing, and rendering steps.  Once those are created, we add them all to the `ModelConfiguration` object and compile them into the `.air` file.

In [4]:
# Harvesting

# For harvesting, we need to harvest using a regular expression
# that identifies possible phone numbers.  The following lines of 
# code configure such harvesting

regex = '/^[\.-)( ]*([0-9]{3})[\.-)( ]*([0-9]{3})[\.-)( ]*([0-9]{4})$/'
harvester = aisquared.config.harvesting.TextHarvester(
    how = 'regex',
    regex = regex
)

In [9]:
# Preprocessing

# The only preprocessing step that has to be conducted is the 
# removal of all punctuation, since the phone numbers in the 
# analytic JSON are all strictly digits

step = aisquared.config.preprocessing.RemoveCharacters(
    remove_digits = False,
    remove_punctuation = True
)
preprocesser = aisquared.config.preprocessing.TextPreprocessor()
preprocesser.add_step(step)

In [11]:
# Analytic

# The analytic for this configuration is going to be a LocalAnalytic
# class, where we pass the saved file to the object

analytic = aisquared.config.analytic.LocalAnalytic(file_name)

In [13]:
# Postprocessing

# No postprocessing steps are needed, so we can set that value to None

postprocesser = None

In [17]:
# Rendering

# To render results, we are going to use the WordRendering class to
# initialize the rendering of badges for individual words. By default,
# the WordRendering class renders the specific words/tokens that were
# input into the analytics

renderer = aisquared.config.rendering.WordRendering()

In [21]:
# Putting it all together

# Finally, we will take the previous objects and put them all 
# together into a single ModelConfiguration object, which is then 
# compiled into the .air file

config = aisquared.config.ModelConfiguration(
    name = 'PhoneNumberLookup',
    harvesting_steps = harvester,
    preprocessing_steps = preprocesser,
    analytic = analytic,
    postprocessing_steps = postprocesser,
    rendering_steps = renderer,
    version = None,
    description = 'Phone number lookup which shows name and customer ID (if present) for phone numbers found',
    mlflow_uri = None,
    mlflow_user = None,
    mlflow_token = None
)

# compile to create .air file
config.compile()